# Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

! pwd

Mounted at /content/drive
/content


# Create Dataset


In [3]:
import os

import cv2


input_dir = '/content/drive/MyDrive/polyp_seg/data/polyDB/masks/train'
output_dir = '/content/drive/MyDrive/polyp_seg/data/polyDB/labels/train'

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    # load the binary mask and get its contours
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

# Install libraries

In [4]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.8/112.6 GB disk)


# Train model

In [10]:
import os
from ultralytics import YOLO


model = YOLO('yolov8n-seg.pt')  # load a pretrained model (recommended for training)

model.train(data='/content/drive/MyDrive/polyp_seg/data/polyDB/config.yaml', epochs=10, imgsz=640)


Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/drive/MyDrive/polyp_seg/data/polyDB/config.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_lab

100%|██████████| 755k/755k [00:00<00:00, 26.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 100MB/s]


AMP: checks passed ✅


...


10 epochs completed in 0.283 hours.
Optimizer stripped from runs/segment/train6/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/train6/weights/best.pt, 6.8MB

Validating runs/segment/train6/weights/best.pt...
Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:14<00:00,  1.31it/s]

                   all        590          0          0          0          0          0          0          0          0          0
WARNING ⚠️ no labels found in segment set, can not compute metrics without labels


Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 4.5ms postprocess per image
Results saved to runs/segment/train6


AttributeError: 'SegmentMetrics' object has no attribute 'curves_results'. See valid attributes below.

    Calculates and aggregates detection and segmentation metrics over a given set of classes.

    Args:
        save_dir (Path): Path to the directory where the output plots should be saved. Default is the current directory.
        plot (bool): Whether to save the detection and segmentation plots. Default is False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (list): List of class names. Default is an empty list.

    Attributes:
        save_dir (Path): Path to the directory where the output plots should be saved.
        plot (bool): Whether to save the detection and segmentation plots.
        on_plot (func): An optional callback to pass plots path and data when they are rendered.
        names (list): List of class names.
        box (Metric): An instance of the Metric class to calculate box det

# Save results

In [12]:
!scp -r /content/runs '/content/drive/MyDrive/polyp_seg/data/polyDB/results'


NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

# Load model and prediction

In [ ]:
from ultralytics import YOLO


In [ ]:
# model = YOLO("/content/gdrive/My Drive/runs")  # load a custom model

# # Predict with the model
# results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image

import cv2

# Save mask only
model_path = '/content/gdrive/My Drive/polyp_seg/best_1.pt'

image_path = '/content/12345.png'

img = cv2.imread(image_path)
H, W, _ = img.shape

model = YOLO(model_path)

results = model(img)
print(f"Image tpye{results}")

for result in results:
    for j, mask in enumerate(result.masks.data):

        mask = mask.cpu().numpy() * 255

        mask = cv2.resize(mask, (W, H))

        cv2.imwrite('./output.png', mask)


0: 640x640 1 polyps, 10.5ms
Speed: 4.0ms preprocess, 10.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Image tpye[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'polyps'}
obb: None
orig_img: array([[[14,  5,  5],
        [14,  5,  5],
        [13,  4,  3],
        ...,
        [ 6,  2,  2],
        [ 6,  2,  2],
        [ 6,  2,  2]],

       [[14,  5,  5],
        [14,  5,  5],
        [13,  4,  3],
        ...,
        [ 6,  2,  2],
        [ 6,  2,  2],
        [ 6,  2,  2]],

       [[14,  5,  5],
        [14,  5,  5],
        [13,  4,  3],
        ...,
        [ 6,  2,  2],
        [ 6,  2,  2],
        [ 6,  2,  2]],

       ...,

       [[11,  2,  2],
        [11,  2,  2],
        [11,  2,  2],
        ...,
        [ 8,  2,  1],
        [ 8,  2,  1],
        [ 8,  2,  1]],

       [[11,  2,  2],
        [11,  2,  2],
       